In [1]:
import pandas as pd
import numpy as np
import re
import datetime as dt
from datetime import timedelta, datetime, time
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras


In [2]:
path = '/content/drive/MyDrive/Aviator/Data1/bd_pseudo_1.csv'
data = pd.read_csv(path)
print(data)
t = 0
print(len(data))
t0 = int(input("Maximo: "))
tamanho_data = len(data)

odd = []
horario = []
odd_ajustada = []

while t <= t0:
    if t < tamanho_data:
        value = data['odd'][t]
        if isinstance(value, float):
            odd.append(np.nan)  # or any other value to represent missing data
        else:
            aux = value.split('\n\n')
            a1 = aux[0].replace(')I', '')
            a2 = a1.replace(')', '')
            a3 = a2.replace('|', '1')
            a4 = a3.replace('?', '7')
            a5 = re.sub('[^0-9.]', '', a4)
            odd.append(float(a5))
        horario.append(data['hora_criacao'][t])

    t += 1

for name in odd:
    ajuste = float(name - 1.25)
    odd_ajustada.append(ajuste)

horas = []
minutos = []
segundos = []
for item in horario:
    dt = datetime.strptime(item, "%a %b %d %H:%M:%S %Y")
    horas.append(dt.hour)
    minutos.append(dt.minute)
    segundos.append(dt.second)

data1 = pd.DataFrame({'Odd_ajustada': odd, 'hora': horas, 'minutos': minutos, 'segundos': segundos, 'horario': horario})

data2 = pd.DataFrame({'horario': horario, 'Odd_ajustada': odd})

tempo = []
for i in range(2, len(data2)):
    #print(data2['horario'][i])
    name = pd.to_datetime(data2['horario'][i - 1]) - pd.to_datetime(data2['horario'][i - 2])
    tempo.append(name)

# Convertendo para segundos
tempo_em_segundos = [t.total_seconds() for t in tempo]
time = np.ones(len(data), dtype = int)
for i in range(0, len(time)):
    if i == 0 or i == 1:
	    time[i] = 12
    else:
    	time[i] = tempo_em_segundos[i - 2]

tempo_em_segundos = time
# Adicionando a coluna de tempo em segundos ao DataFrame
data1['tempo_em_segundos'] = tempo_em_segundos

dados_filtrados = data2[data2['Odd_ajustada'] < 20]

###Data final
#data = dados_filtrados
data = data1
data2 = data1

# Verificar se existem valores NaN na coluna "Odd_ajustada"
has_nan = data['Odd_ajustada'].isna().any()
if has_nan:
    print("Existem valores NaN na coluna 'Odd_ajustada'.")
else:
    print("Não existem valores NaN na coluna 'Odd_ajustada'.")


          id    odd              hora_criacao apostadores
0          0  13.89  Sat May 27 00:28:06 2023        1650
1          1   1.44  Sat May 27 00:28:18 2023       22211
2          2   3.39  Sat May 27 00:28:42 2023        1751
3          3   1.27  Sat May 27 00:28:56 2023        1950
4          4   5.16  Sat May 27 00:29:26 2023        2086
...      ...    ...                       ...         ...
35400  35403   1.99  Tue Oct 17 21:51:38 2023       2992
35401  35404   4.17  Tue Oct 17 21:52:12 2023       2040
35402  35405  15.63  Tue Oct 17 21:52:56 2023           
35403  35406   1.65  Tue Oct 17 21:53:15 2023       2496
35404  35407  10.97  Tue Oct 17 21:53:56 2023        200

[35405 rows x 4 columns]
35405
Maximo: 35405
Não existem valores NaN na coluna 'Odd_ajustada'.


In [3]:
data

,Odd_ajustada,hora,minutos,segundos,horario,tempo_em_segundos
0,13.89,0,28,6,Sat May 27 00:28:06 2023,12
1,1.44,0,28,18,Sat May 27 00:28:18 2023,12
2,3.39,0,28,42,Sat May 27 00:28:42 2023,12
3,1.27,0,28,56,Sat May 27 00:28:56 2023,24
4,5.16,0,29,26,Sat May 27 00:29:26 2023,14
...,...,...,...,...,...,...
35400,1.99,21,51,38,Tue Oct 17 21:51:38 2023,26
35401,4.17,21,52,12,Tue Oct 17 21:52:12 2023,21
35402,15.63,21,52,56,Tue Oct 17 21:52:56 2023,34
35403,1.65,21,53,15,Tue Oct 17 21:53:15 2023,44


In [4]:
# Crie colunas separadas para o dia, mês e ano
data['dia'] = pd.to_datetime(data['horario']).dt.day
data['mes'] = pd.to_datetime(data['horario']).dt.month
data['ano'] = pd.to_datetime(data['horario']).dt.year

# Suponhamos que sua coluna de data e hora seja chamada 'entrada'
data['horario'] = pd.to_datetime(data['horario']).dt.time

In [5]:
data

,Odd_ajustada,hora,minutos,segundos,horario,tempo_em_segundos,dia,mes,ano
0,13.89,0,28,6,00:28:06,12,27,5,2023
1,1.44,0,28,18,00:28:18,12,27,5,2023
2,3.39,0,28,42,00:28:42,12,27,5,2023
3,1.27,0,28,56,00:28:56,24,27,5,2023
4,5.16,0,29,26,00:29:26,14,27,5,2023
...,...,...,...,...,...,...,...,...,...
35400,1.99,21,51,38,21:51:38,26,17,10,2023
35401,4.17,21,52,12,21:52:12,21,17,10,2023
35402,15.63,21,52,56,21:52:56,34,17,10,2023
35403,1.65,21,53,15,21:53:15,44,17,10,2023


In [6]:
t = len(data)
total = []
for i in range(0,t):
  #print(data['hora'][i], data['minutos'][i], data['segundos'][i])
  hour = data['hora'][i]*3600
  minutes = data['minutos'][i]*60
  soma = hour + minutes + data['segundos'][i]
  total.append(soma)
total = pd.DataFrame(total, columns = ['total_segundos'])
print(total)
data = pd.concat([data, total], axis = 1)

       total_segundos
0                1686
1                1698
2                1722
3                1736
4                1766
...               ...
35400           78698
35401           78732
35402           78776
35403           78795
35404           78836

[35405 rows x 1 columns]


In [7]:
data

,Odd_ajustada,hora,minutos,segundos,horario,tempo_em_segundos,dia,mes,ano,total_segundos
0,13.89,0,28,6,00:28:06,12,27,5,2023,1686
1,1.44,0,28,18,00:28:18,12,27,5,2023,1698
2,3.39,0,28,42,00:28:42,12,27,5,2023,1722
3,1.27,0,28,56,00:28:56,24,27,5,2023,1736
4,5.16,0,29,26,00:29:26,14,27,5,2023,1766
...,...,...,...,...,...,...,...,...,...,...
35400,1.99,21,51,38,21:51:38,26,17,10,2023,78698
35401,4.17,21,52,12,21:52:12,21,17,10,2023,78732
35402,15.63,21,52,56,21:52:56,34,17,10,2023,78776
35403,1.65,21,53,15,21:53:15,44,17,10,2023,78795


In [8]:
data1 = data
data = pd.DataFrame()
data['Odd_ajustada'] = data1['Odd_ajustada']
data['horario'] = data1['horario']
data['total_segundos']  = data1['total_segundos']

In [9]:
data

,Odd_ajustada,horario,total_segundos
0,13.89,00:28:06,1686
1,1.44,00:28:18,1698
2,3.39,00:28:42,1722
3,1.27,00:28:56,1736
4,5.16,00:29:26,1766
...,...,...,...
35400,1.99,21:51:38,78698
35401,4.17,21:52:12,78732
35402,15.63,21:52:56,78776
35403,1.65,21:53:15,78795


In [10]:
# Crie as colunas "entrada" e "saida" deslocando a coluna "horario" para cima
new_data = pd.DataFrame()
new_data['saida'] = data['total_segundos']
new_data['entrada'] = data['total_segundos'].shift(1)

new_data = new_data.drop(new_data.index[0])
# Visualize o DataFrame new_data
print(new_data)

       saida  entrada
1       1698   1686.0
2       1722   1698.0
3       1736   1722.0
4       1766   1736.0
5       1790   1766.0
...      ...      ...
35400  78698  78677.0
35401  78732  78698.0
35402  78776  78732.0
35403  78795  78776.0
35404  78836  78795.0

[35404 rows x 2 columns]


In [11]:
data = pd.concat([data, new_data], axis = 1)
data

,Odd_ajustada,horario,total_segundos,saida,entrada
0,13.89,00:28:06,1686,NaN,NaN
1,1.44,00:28:18,1698,1698.0,1686.0
2,3.39,00:28:42,1722,1722.0,1698.0
3,1.27,00:28:56,1736,1736.0,1722.0
4,5.16,00:29:26,1766,1766.0,1736.0
...,...,...,...,...,...
35400,1.99,21:51:38,78698,78698.0,78677.0
35401,4.17,21:52:12,78732,78732.0,78698.0
35402,15.63,21:52:56,78776,78776.0,78732.0
35403,1.65,21:53:15,78795,78795.0,78776.0


In [12]:
data = data.drop(data.index[0])

In [13]:
data

,Odd_ajustada,horario,total_segundos,saida,entrada
1,1.44,00:28:18,1698,1698.0,1686.0
2,3.39,00:28:42,1722,1722.0,1698.0
3,1.27,00:28:56,1736,1736.0,1722.0
4,5.16,00:29:26,1766,1766.0,1736.0
5,2.65,00:29:50,1790,1790.0,1766.0
...,...,...,...,...,...
35400,1.99,21:51:38,78698,78698.0,78677.0
35401,4.17,21:52:12,78732,78732.0,78698.0
35402,15.63,21:52:56,78776,78776.0,78732.0
35403,1.65,21:53:15,78795,78795.0,78776.0


In [ ]:
# Converta as colunas "entrada" e "saida" em objetos datetime
data['entrada'] = pd.to_datetime(data['entrada'], format='%H:%M:%S')
data['saida'] = pd.to_datetime(data['saida'], format='%H:%M:%S')

# Extraia o horário como uma coluna separada
data['horario_entrada'] = data['entrada'].dt.strftime('%H:%M:%S')
data['horario_saida'] = data['saida'].dt.strftime('%H:%M:%S')

# Converta os horários em segundos
data['entrada'] = data['entrada'].dt.hour * 3600 + data['entrada'].dt.minute * 60 + data['entrada'].dt.second
data['saida'] = data['saida'].dt.hour * 3600 + data['saida'].dt.minute * 60 + data['saida'].dt.second


In [14]:

# Normalize os dados
scaler = StandardScaler()
X = data[['entrada']].values
y = data[['saida']].values
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

# Divida os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crie o modelo de rede neural
model = keras.Sequential([
    keras.layers.LSTM(128, activation='relu', input_shape=(1, 1), kernel_regularizer=l2(0.001)),
    keras.layers.Dense(128, activation='relu', input_shape=(X.shape[1],)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1)
])


# Compile o modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

# Treine o modelo
model.fit(X_train, y_train, epochs=50, batch_size=64)

# Avalie o modelo no conjunto de teste
loss = model.evaluate(X_test, y_test)
print(f"Erro Médio Quadrático no conjunto de teste: {loss}")

# Faça previsões no conjunto de teste
y_pred = model.predict(X_test)

# Desnormalize as previsões
y_pred = scaler.inverse_transform(y_pred)


Epoch 1/50
443/443 [==============================] - 5s 7ms/step - loss: 0.0473 - mean_squared_error: 0.0459
Epoch 2/50
443/443 [==============================] - 4s 9ms/step - loss: 0.0050 - mean_squared_error: 0.0042
Epoch 3/50
443/443 [==============================] - 3s 6ms/step - loss: 0.0048 - mean_squared_error: 0.0042
Epoch 4/50
443/443 [==============================] - 3s 6ms/step - loss: 0.0047 - mean_squared_error: 0.0042
Epoch 5/50
443/443 [==============================] - 3s 6ms/step - loss: 0.0048 - mean_squared_error: 0.0044
Epoch 6/50
443/443 [==============================] - 3s 7ms/step - loss: 0.0046 - mean_squared_error: 0.0042
Epoch 7/50
443/443 [==============================] - 4s 9ms/step - loss: 0.0046 - mean_squared_error: 0.0043
Epoch 8/50
443/443 [==============================] - 3s 6ms/step - loss: 0.0045 - mean_squared_error: 0.0042
Epoch 9/50
443/443 [==============================] - 3s 6ms/step - loss: 0.0045 - mean_squared_error: 0.0042
Epoch 10/5

In [15]:
print(y_pred)

[[51572.754]
 [56108.07 ]
 [13098.415]
 ...
 [52839.33 ]
 [78082.04 ]
 [16233.778]]


In [16]:
# Converter de segundos para o formato de horário
y_pred_horario = pd.to_datetime(y_pred[:, 0], unit='s').time

In [17]:
y_pred_horario

array([datetime.time(14, 19, 32, 753906), datetime.time(15, 35, 8, 70312),
       datetime.time(3, 38, 18, 415039), ...,
       datetime.time(14, 40, 39, 328125),
       datetime.time(21, 41, 22, 39062), datetime.time(4, 30, 33, 778320)],
      dtype=object)

In [18]:
y_pred_horario_str = [horario.strftime('%H:%M:%S') for horario in y_pred_horario]

In [19]:
y_pred_horario_str

['14:19:32',
 '15:35:08',
 '03:38:18',
 '18:28:10',
 '14:10:03',
 '21:02:36',
 '05:47:13',
 '01:46:11',
 '19:15:29',
 '03:00:05',
 '18:20:02',
 '08:03:57',
 '22:11:24',
 '05:06:22',
 '01:38:48',
 '17:13:25',
 '19:37:22',
 '03:21:57',
 '20:41:46',
 '18:58:18',
 '13:51:59',
 '04:48:51',
 '14:33:30',
 '04:42:21',
 '00:14:34',
 '16:42:05',
 '13:19:13',
 '17:02:34',
 '05:08:09',
 '21:52:46',
 '13:59:29',
 '17:07:25',
 '17:59:59',
 '16:12:03',
 '16:05:45',
 '15:15:38',
 '12:49:34',
 '15:08:16',
 '03:01:58',
 '13:28:25',
 '00:50:52',
 '13:15:38',
 '05:15:07',
 '02:35:50',
 '16:51:30',
 '00:22:47',
 '03:28:59',
 '18:13:32',
 '00:47:35',
 '19:49:10',
 '23:50:00',
 '02:34:45',
 '08:58:50',
 '17:02:53',
 '04:49:59',
 '13:05:20',
 '19:34:04',
 '06:55:25',
 '20:31:06',
 '05:48:01',
 '11:20:18',
 '04:53:02',
 '15:31:20',
 '19:25:58',
 '11:24:48',
 '13:36:54',
 '12:37:45',
 '06:13:51',
 '04:17:01',
 '15:53:32',
 '11:31:02',
 '13:31:40',
 '17:23:37',
 '14:15:15',
 '16:52:07',
 '19:26:19',
 '23:17:39',

In [20]:
data

,Odd_ajustada,horario,total_segundos,saida,entrada
1,1.44,00:28:18,1698,1698.0,1686.0
2,3.39,00:28:42,1722,1722.0,1698.0
3,1.27,00:28:56,1736,1736.0,1722.0
4,5.16,00:29:26,1766,1766.0,1736.0
5,2.65,00:29:50,1790,1790.0,1766.0
...,...,...,...,...,...
35400,1.99,21:51:38,78698,78698.0,78677.0
35401,4.17,21:52:12,78732,78732.0,78698.0
35402,15.63,21:52:56,78776,78776.0,78732.0
35403,1.65,21:53:15,78795,78795.0,78776.0


In [21]:
print(data['entrada'].tail(1).reset_index(drop = True).iloc[0])
new_entry = pd.DataFrame()
new_entry['entrada'] = data['entrada'].tail(1).reset_index(drop = True)

new_entry = new_entry[['entrada']].values
new_entry = scaler.fit_transform(new_entry)

y_pred_new = model.predict(new_entry)

print(y_pred_new)

y_pred_hor = scaler.inverse_transform(y_pred_new)

print(y_pred_hor)

78795.0
1/1 [==============================] - 0s 37ms/step
[[0.00087596]]
[[78795.]]


In [25]:
# Converter de segundos para o formato de horário
source = y_pred_hor[:, 0] + 12
y_pred_horario = pd.to_datetime(source, unit='s').time
y_pred_horario_str = [horario.strftime('%H:%M:%S') for horario in y_pred_horario]

In [26]:
y_pred_horario

array([datetime.time(21, 53, 27)], dtype=object)

In [27]:
from datetime import time

# Obtenha o valor do array
horario_predito = y_pred_horario[0]

# Converta o valor para um objeto de tempo
horario_obj = time(
    horario_predito.hour,  # Extraia as horas
    horario_predito.minute,  # Extraia os minutos
    horario_predito.second,  # Extraia os segundos
    horario_predito.microsecond  # Extraia os microssegundos (opcional)
)

# Agora você tem o horário em um formato de objeto de tempo
print(horario_obj)


21:53:27


In [28]:
data

,Odd_ajustada,horario,total_segundos,saida,entrada
1,1.44,00:28:18,1698,1698.0,1686.0
2,3.39,00:28:42,1722,1722.0,1698.0
3,1.27,00:28:56,1736,1736.0,1722.0
4,5.16,00:29:26,1766,1766.0,1736.0
5,2.65,00:29:50,1790,1790.0,1766.0
...,...,...,...,...,...
35400,1.99,21:51:38,78698,78698.0,78677.0
35401,4.17,21:52:12,78732,78732.0,78698.0
35402,15.63,21:52:56,78776,78776.0,78732.0
35403,1.65,21:53:15,78795,78795.0,78776.0


In [37]:
k = 0
array = []
print(data['entrada'].tail(1).reset_index(drop = True).iloc[0])
new_entry = pd.DataFrame()
new_entry['entrada'] = data['entrada'].tail(1).reset_index(drop = True)

while k <= 150:

  new_entry = new_entry[['entrada']].values
  new_entry = scaler.fit_transform(new_entry)

  y_pred_new = model.predict(new_entry)

  print(y_pred_new)

  y_pred_hor = scaler.inverse_transform(y_pred_new)

  print(y_pred_hor)
  # Converter de segundos para o formato de horário
  source = y_pred_hor[:, 0] + 27.5
  y_pred_horario = pd.to_datetime(source, unit='s').time
  y_pred_horario_str = [horario.strftime('%H:%M:%S') for horario in y_pred_horario]

  y_pred_horario


  # Obtenha o valor do array
  horario_predito = y_pred_horario[0]

  # Converta o valor para um objeto de tempo
  horario_obj = time(
      horario_predito.hour,  # Extraia as horas
      horario_predito.minute,  # Extraia os minutos
      horario_predito.second,  # Extraia os segundos
      horario_predito.microsecond  # Extraia os microssegundos (opcional)
  )

  # Agora você tem o horário em um formato de objeto de tempo
  print(horario_obj)
  array.append(horario_obj)

  new_entry = pd.DataFrame([source], columns = ['entrada'])
  k += 1

78795.0
1/1 [==============================] - 0s 33ms/step
[[0.00087596]]
[[78795.]]
21:53:42.500000
1/1 [==============================] - 0s 60ms/step
[[0.00087596]]
[[78822.5]]
21:54:10
1/1 [==============================] - 0s 114ms/step
[[0.00087596]]
[[78850.]]
21:54:37.500000
1/1 [==============================] - 0s 45ms/step
[[0.00087596]]
[[78877.5]]
21:55:05
1/1 [==============================] - 0s 88ms/step
[[0.00087596]]
[[78905.]]
21:55:32.500000
1/1 [==============================] - 0s 74ms/step
[[0.00087596]]
[[78932.5]]
21:56:00
1/1 [==============================] - 0s 33ms/step
[[0.00087596]]
[[78960.]]
21:56:27.500000
1/1 [==============================] - 0s 44ms/step
[[0.00087596]]
[[78987.5]]
21:56:55
1/1 [==============================] - 0s 34ms/step
[[0.00087596]]
[[79015.]]
21:57:22.500000
1/1 [==============================] - 0s 38ms/step
[[0.00087596]]
[[79042.5]]
21:57:50
1/1 [==============================] - 0s 39ms/step
[[0.00087596]]
[[79070.]]
21:

In [33]:
array

[datetime.time(21, 53, 27),
 datetime.time(21, 53, 39),
 datetime.time(21, 53, 51),
 datetime.time(21, 54, 3),
 datetime.time(21, 54, 15),
 datetime.time(21, 54, 27),
 datetime.time(21, 54, 39),
 datetime.time(21, 54, 51),
 datetime.time(21, 55, 3),
 datetime.time(21, 55, 15),
 datetime.time(21, 55, 27),
 datetime.time(21, 55, 39),
 datetime.time(21, 55, 51),
 datetime.time(21, 56, 3),
 datetime.time(21, 56, 15),
 datetime.time(21, 56, 27),
 datetime.time(21, 56, 39),
 datetime.time(21, 56, 51),
 datetime.time(21, 57, 3),
 datetime.time(21, 57, 15),
 datetime.time(21, 57, 27),
 datetime.time(21, 57, 39),
 datetime.time(21, 57, 51),
 datetime.time(21, 58, 3),
 datetime.time(21, 58, 15),
 datetime.time(21, 58, 27),
 datetime.time(21, 58, 39),
 datetime.time(21, 58, 51),
 datetime.time(21, 59, 3),
 datetime.time(21, 59, 15),
 datetime.time(21, 59, 27),
 datetime.time(21, 59, 39),
 datetime.time(21, 59, 51),
 datetime.time(22, 0, 3),
 datetime.time(22, 0, 15),
 datetime.time(22, 0, 27),
 d

In [101]:
data2 = data1
data = data1

In [102]:
data

,Odd_ajustada,hora,minutos,segundos,horario,tempo_em_segundos,dia,mes,ano,total_segundos
0,13.89,0,28,6,00:28:06,12,27,5,2023,1686
1,1.44,0,28,18,00:28:18,12,27,5,2023,1698
2,3.39,0,28,42,00:28:42,12,27,5,2023,1722
3,1.27,0,28,56,00:28:56,24,27,5,2023,1736
4,5.16,0,29,26,00:29:26,14,27,5,2023,1766
...,...,...,...,...,...,...,...,...,...,...
35400,1.99,21,51,38,21:51:38,26,17,10,2023,78698
35401,4.17,21,52,12,21:52:12,21,17,10,2023,78732
35402,15.63,21,52,56,21:52:56,34,17,10,2023,78776
35403,1.65,21,53,15,21:53:15,44,17,10,2023,78795


In [103]:
data = pd.DataFrame()
data['Odd_ajustada'] = data1['Odd_ajustada']
data['horario'] = data1['horario']
data['total_segundos']  = data1['total_segundos']

# Crie as colunas "entrada" e "saida" deslocando a coluna "horario" para cima
new_data = pd.DataFrame()
new_data['saida'] = data['total_segundos']
new_data['entrada'] = data['total_segundos'].shift(1)

new_data = new_data.drop(new_data.index[0])
# Visualize o DataFrame new_data
print(new_data)

data = pd.concat([data, new_data], axis = 1)

data = data.drop(data.index[0])

print(data)

       saida  entrada
1       1698   1686.0
2       1722   1698.0
3       1736   1722.0
4       1766   1736.0
5       1790   1766.0
...      ...      ...
35400  78698  78677.0
35401  78732  78698.0
35402  78776  78732.0
35403  78795  78776.0
35404  78836  78795.0

[35404 rows x 2 columns]
       Odd_ajustada   horario  total_segundos    saida  entrada
1              1.44  00:28:18            1698   1698.0   1686.0
2              3.39  00:28:42            1722   1722.0   1698.0
3              1.27  00:28:56            1736   1736.0   1722.0
4              5.16  00:29:26            1766   1766.0   1736.0
5              2.65  00:29:50            1790   1790.0   1766.0
...             ...       ...             ...      ...      ...
35400          1.99  21:51:38           78698  78698.0  78677.0
35401          4.17  21:52:12           78732  78732.0  78698.0
35402         15.63  21:52:56           78776  78776.0  78732.0
35403          1.65  21:53:15           78795  78795.0  78776.0
35404 

In [56]:
data['tempo_media_movel5'] = data['entrada'].rolling(window=5).mean()
data['tempo_media_movel10'] = data['entrada'].rolling(window=10).mean()
data['tempo_media_movel20'] = data['entrada'].rolling(window=20).mean()
data['tempo_media_movel40'] = data['entrada'].rolling(window=40).mean()
data['tempo_media_movel80'] = data['entrada'].rolling(window=80).mean()
data['tempo_media_movel160'] = data['entrada'].rolling(window=160).mean()
data['tempo_media_movel320'] = data['entrada'].rolling(window=320).mean()
data['tempo_media_movel640'] = data['entrada'].rolling(window=640).mean()

# Estatísticas Descritivas
data['tempo_desvio_padrao5'] = data['entrada'].rolling(window=5).std()
data['tempo_desvio_padrao10'] = data['entrada'].rolling(window=10).std()
data['tempo_desvio_padrao20'] = data['entrada'].rolling(window=20).std()
data['tempo_desvio_padrao40'] = data['entrada'].rolling(window=40).std()
data['tempo_desvio_padrao80'] = data['entrada'].rolling(window=80).std()
data['tempo_desvio_padrao160'] = data['entrada'].rolling(window=160).std()
data['tempo_desvio_padrao320'] = data['entrada'].rolling(window=320).std()
data['tempo_desvio_padrao640'] = data['entrada'].rolling(window=640).std()

data = data.fillna(data.mean())  # Preencher com a média da coluna


<ipython-input-56-369c84ab786d>:20: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data = data.fillna(data.mean())  # Preencher com a média da coluna


In [57]:
data.columns

Index(['Odd_ajustada', 'horario', 'total_segundos', 'saida', 'entrada',
       'tempo_media_movel5', 'tempo_media_movel10', 'tempo_media_movel20',
       'tempo_media_movel40', 'tempo_media_movel80', 'tempo_media_movel160',
       'tempo_media_movel320', 'tempo_media_movel640', 'tempo_desvio_padrao5',
       'tempo_desvio_padrao10', 'tempo_desvio_padrao20',
       'tempo_desvio_padrao40', 'tempo_desvio_padrao80',
       'tempo_desvio_padrao160', 'tempo_desvio_padrao320',
       'tempo_desvio_padrao640'],
      dtype='object')

In [59]:
scaler = StandardScaler()
X = data[['entrada']].values
y = data[['saida']].values
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='linear')  # Camada de saída para regressão
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

model.fit(X_train, y_train, epochs=50, batch_size=32)

loss = model.evaluate(X_test, y_test)
print(f"Erro Médio Quadrático no conjunto de teste: {loss}")

y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)


Epoch 1/50
886/886 [==============================] - 2s 2ms/step - loss: 0.0165 - mean_squared_error: 0.0165
Epoch 2/50
886/886 [==============================] - 2s 2ms/step - loss: 0.0068 - mean_squared_error: 0.0068
Epoch 3/50
886/886 [==============================] - 2s 2ms/step - loss: 0.0090 - mean_squared_error: 0.0090
Epoch 4/50
886/886 [==============================] - 2s 2ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 5/50
886/886 [==============================] - 2s 2ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 6/50
886/886 [==============================] - 2s 3ms/step - loss: 0.0054 - mean_squared_error: 0.0054
Epoch 7/50
886/886 [==============================] - 2s 2ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 8/50
886/886 [==============================] - 2s 2ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 9/50
886/886 [==============================] - 2s 2ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 10/5

In [105]:
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Normalize the data
scaler = StandardScaler()
X = data[['entrada']].values
y = data[['saida']].values
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

# Divida os dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the model
model = keras.Sequential()

# Add LSTM layers with dropout
model.add(LSTM(64, activation='relu', input_shape=(X.shape[1], 1), return_sequences=True, kernel_regularizer=l2(0.001)))
model.add(Dropout(0.2))  # Add dropout to the first LSTM layer

# You can experiment with adding more LSTM layers here
model.add(LSTM(64, activation='relu', return_sequences=True, kernel_regularizer=l2(0.001)))
model.add(Dropout(0.2))  # Add dropout to the second LSTM layer

# Add a final LSTM layer without return_sequences
model.add(LSTM(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.2))  # Add dropout to the final LSTM layer


# Add fully connected layers
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dense(1, activation='linear'))  # Output layer for regression

# Compile the model with the Adam optimizer
adam_optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=adam_optimizer, loss='mean_squared_error', metrics=['mean_squared_error'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32)

# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f"Erro Médio Quadrático no conjunto de teste: {loss}")

y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)


Epoch 1/50
886/886 [==============================] - 13s 6ms/step - loss: 0.1672 - mean_squared_error: 0.1193
Epoch 2/50
886/886 [==============================] - 7s 8ms/step - loss: 0.0557 - mean_squared_error: 0.0299
Epoch 3/50
886/886 [==============================] - 6s 7ms/step - loss: 0.0487 - mean_squared_error: 0.0261
Epoch 4/50
886/886 [==============================] - 6s 7ms/step - loss: 0.0444 - mean_squared_error: 0.0237
Epoch 5/50
886/886 [==============================] - 7s 8ms/step - loss: 0.0418 - mean_squared_error: 0.0227
Epoch 6/50
886/886 [==============================] - 6s 6ms/step - loss: 0.0395 - mean_squared_error: 0.0217
Epoch 7/50
886/886 [==============================] - 7s 8ms/step - loss: 0.0369 - mean_squared_error: 0.0203
Epoch 8/50
886/886 [==============================] - 6s 6ms/step - loss: 0.0348 - mean_squared_error: 0.0192
Epoch 9/50
886/886 [==============================] - 7s 8ms/step - loss: 0.0332 - mean_squared_error: 0.0184
Epoch 10/

In [106]:
# Converter de segundos para o formato de horário
y_pred_horario = pd.to_datetime(y_pred[:, 0], unit='s').time

In [107]:
y_pred_horario

array([datetime.time(14, 23, 36, 363281),
       datetime.time(15, 27, 53, 707031),
       datetime.time(3, 11, 58, 692382), ...,
       datetime.time(14, 41, 28, 804687),
       datetime.time(21, 25, 7, 625000), datetime.time(4, 19, 42, 924804)],
      dtype=object)

In [108]:
data.columns

Index(['Odd_ajustada', 'horario', 'total_segundos', 'saida', 'entrada'], dtype='object')

In [109]:
print(data['entrada'].tail(1).reset_index(drop=True).iloc[0])
new_entry = data.tail(1).reset_index(drop=True)
new_entry = new_entry[['entrada',]].values
new_entry = scaler.transform(new_entry)  # Use transform instead of fit_transform

y_pred_new = model.predict(new_entry)

print(y_pred_new)

# Reshape y_pred_new to match the original shape
#y_pred_new = y_pred_new.reshape(1, 1)  # Reshape to (1, 17)

# Inverse transform the prediction
y_pred_hor = scaler.inverse_transform(y_pred_new)
print(y_pred_hor)


78795.0
1/1 [==============================] - 0s 37ms/step
[[1.2716926]]
[[77770.07]]


In [114]:
y_pred_horario = pd.to_datetime(y_pred_hor[:, 0], unit='s').time
y_pred_horario_str = [horario.strftime('%H:%M:%S') for horario in y_pred_horario]

In [115]:
y_pred_horario_str

['21:36:10']